<a href="https://colab.research.google.com/github/CamiloVga/Curso-IA-Para-Ciencia-de-Datos/blob/main/Script_Sesi%C3%B3n_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IA para la Ciencia de Datos
## Universidad de los Andes

**Profesor:** Camilo Vega - AI/ML Engineer  
**LinkedIn:** https://www.linkedin.com/in/camilo-vega-169084b1/

---

## Guía: GenBI - Interfaces Conversacionales de Datos con LLM

Este notebook presenta **2 implementaciones prácticas**:

1. **RAG con Gradio** - Chat de documentos con interfaz simple
2. **Text-to SQL y MatplotLib y Seaborn** - Consultas naturales a bases de datos

### Requisitos
- **APIs:** Groq API token
- **GPU:** Opcional para modelos locales
- **Datos:** Documentos PDF/DOCX/TXT/CSV para subir

## Configuración APIs
- **Groq API:**
  1. [Crear token](https://console.groq.com/keys)
  2. En Colab: 🔑 Secrets → Agregar `GROQ_KEY` → Pegar tu token

Cada sección es **independiente** y puede ejecutarse por separado

#Gradio para RAG con Documentos

In [ ]:
# --- 1. Instalación de Dependencias ---
!pip install gradio groq scikit-learn PyPDF2 python-docx pandas openpyxl -q

# --- 2. Importación de Librerías ---
import os
import gradio as gr
import pandas as pd
import numpy as np
from groq import Groq
from google.colab import userdata
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from PyPDF2 import PdfReader
from docx import Document

# --- 3. Configuración Global ---
try:
    GROQ_API_KEY = userdata.get('GROQ_KEY')
    client = Groq(api_key=GROQ_API_KEY)
    GROQ_MODEL = "llama-3.1-8b-instant"
except (ImportError, KeyError):
    print("ADVERTENCIA: No se encontró la clave de API de Groq.")
    client = None
    GROQ_MODEL = None


# --- 4. Clase Principal del Motor RAG ---
class DocumentRAG:
    """Encapsula la lógica para procesar documentos, crear vectores y generar respuestas."""
    def __init__(self):
        self.vectorizer = TfidfVectorizer(max_features=2000, ngram_range=(1, 2), min_df=2)
        self.chunks = []
        self.vectors = None
        self.is_ready = False

    def load_documents(self, files):
        if not files: return "No se subieron archivos."
        combined_text = ""
        processed_files = []
        for file in files:
            filename = os.path.basename(file.name)
            try:
                if filename.endswith('.txt'):
                    with open(file.name, 'r', encoding='utf-8', errors='ignore') as f:
                        combined_text += f.read() + "\n\n"
                elif filename.endswith('.pdf'):
                    reader = PdfReader(file.name)
                    for page in reader.pages: combined_text += (page.extract_text() or "") + "\n"
                elif filename.endswith('.docx'):
                    doc = Document(file.name)
                    for p in doc.paragraphs: combined_text += p.text + "\n"
                elif filename.endswith(('.csv', '.xlsx')):
                    df = pd.read_csv(file.name) if filename.endswith('.csv') else pd.read_excel(file.name)
                    combined_text += df.to_string() + "\n\n"
                processed_files.append(filename)
            except Exception:
                continue
        if combined_text.strip():
            self._create_knowledge_base(combined_text)
            return f"✅ {len(processed_files)} archivos procesados. {len(self.chunks)} fragmentos de conocimiento creados."
        return "❌ No se pudo extraer texto de los archivos."

    def _create_knowledge_base(self, text):
        text = text.replace('\n', ' ').strip()
        sentences = text.split('.')
        current_chunk = ""
        for sentence in sentences:
            if len(current_chunk) + len(sentence) < 1000:
                current_chunk += sentence + ". "
            else:
                if len(current_chunk) > 150: self.chunks.append(current_chunk.strip())
                current_chunk = sentence + ". "
        if len(current_chunk) > 150: self.chunks.append(current_chunk.strip())
        if self.chunks:
            self.vectors = self.vectorizer.fit_transform(self.chunks)
            self.is_ready = True

    def get_response(self, message):
        if not self.is_ready or not client:
            return "El sistema no está listo. Por favor, carga documentos primero."
        query_vector = self.vectorizer.transform([message])
        similarities = cosine_similarity(query_vector, self.vectors)[0]
        top_indices = [idx for idx in np.argsort(similarities)[-3:][::-1] if similarities[idx] > 0.05]
        if not top_indices:
            return "No encontré información relevante para tu pregunta."
        context = "\n\n---\n\n".join([self.chunks[idx] for idx in top_indices])
        prompt = f"Basándote en este contexto:\n\n{context}\n\nPregunta: {message}\n\nResponde en español, de forma concisa y solo con la información proporcionada."
        try:
            response = client.chat.completions.create(
                model=GROQ_MODEL, messages=[{"role": "user", "content": prompt}],
                max_tokens=500, temperature=0.1)
            return response.choices[0].message.content
        except Exception as e:
            return f"Error con el modelo de lenguaje: {e}"

# --- 5. Funciones de la Interfaz Gradio ---
def handle_file_upload(files, rag_state):
    rag_state = DocumentRAG()
    status = rag_state.load_documents(files)
    return status, rag_state

def get_chat_response(message, history, rag_state):
    if not rag_state.is_ready:
        return "Por favor, carga primero algunos documentos para empezar a chatear."
    return rag_state.get_response(message)

# --- 6. Creación de la Interfaz de Usuario ---
def create_interface():
    with gr.Blocks(theme=gr.themes.Soft(), title="Chat de Documentos") as app:
        rag_state = gr.State(DocumentRAG())
        with gr.Row():
            with gr.Column(scale=1):
                gr.Markdown("### 1. Carga tus Documentos")
                file_upload = gr.File(label="Sube archivos (.pdf, .docx, .txt)", file_count="multiple")
                upload_status = gr.Textbox(label="Estado", interactive=False)
            with gr.Column(scale=2):
                gr.Markdown("### 2. Chatea con ellos")
                chat_interface = gr.ChatInterface(
                    fn=get_chat_response,
                    additional_inputs=[rag_state],
                    examples=[
                        ["¿Cuál es el tema principal de los documentos?"],
                        ["Resume los puntos clave en tres viñetas"],
                        ["¿Hay alguna conclusión importante?"]
                    ]
                )
        file_upload.change(
            fn=handle_file_upload,
            inputs=[file_upload, rag_state],
            outputs=[upload_status, rag_state]
        )
    return app

# --- 8. Lanzamiento de la Aplicación ---
if __name__ == "__main__":
    app = create_interface()
    app.launch(share=True, debug=True)

#Gradio para RAG Base Datos y Gráficas

In [ ]:
# --- 1. Instalación de Dependencias ---
# Instala todas las librerías necesarias para el funcionamiento del script.
!pip install groq sentence-transformers pandas numpy scikit-learn gradio matplotlib seaborn plotly openpyxl -q

# --- 2. Importación de Librerías ---
import pandas as pd
import numpy as np
import sqlite3
import warnings
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from groq import Groq
import gradio as gr
import plotly.express as px
from google.colab import userdata

warnings.filterwarnings('ignore')

# --- 3. Configuración Global ---
# Define constantes y configuraciones que se utilizarán en toda la aplicación.

# Clave de API para el modelo de lenguaje (se obtiene de los secrets de Google Colab).
try:
    GROQ_API_KEY = userdata.get('GROQ_KEY')
    client = Groq(api_key=GROQ_API_KEY)
except (ImportError, KeyError):
    print("ADVERTENCIA: No se encontró la clave de API de Groq. El sistema funcionará sin LLM.")
    client = None

# Modelos a utilizar.
GROQ_MODEL = "llama-3.1-8b-instant"
EMBEDDING_MODEL = SentenceTransformer('all-MiniLM-L6-v2')

# Constantes del sistema.
TABLE_NAME = 'data'
MAX_ROWS_FOR_ANALYSIS = 5000 # Límite para visualización en gráficas para no saturar el navegador.


# --- 4. Clase Principal del Sistema RAG ---
class DatabaseRAG:
    """
    Encapsula toda la lógica para cargar datos, generar SQL, ejecutar consultas
    y crear respuestas. Cada sesión de usuario tendrá su propia instancia de esta clase.
    """

    def __init__(self):
        """Constructor: Inicializa las variables de estado para una sesión."""
        self.conn = None
        self.column_embeddings = {}
        self.column_info = {}
        self.schema_description = ""
        self.dataframe_head = pd.DataFrame()

    def load_file(self, file_path):
        """Carga un archivo (CSV, XLSX, JSON) en una base de datos SQLite en memoria."""
        try:
            # Lee el archivo según su extensión.
            if file_path.lower().endswith('.csv'):
                df = pd.read_csv(file_path)
            elif file_path.lower().endswith(('.xlsx', '.xls')):
                df = pd.read_excel(file_path)
            elif file_path.lower().endswith('.json'):
                df = pd.read_json(file_path)
            else:
                return False, "Formato no soportado. Use CSV, XLSX o JSON."

            if df.empty:
                return False, "El archivo está vacío."

            # Crea la base de datos en memoria y carga los datos.
            self.conn = sqlite3.connect(':memory:', check_same_thread=False)
            df.to_sql(TABLE_NAME, self.conn, index=False, if_exists='replace')

            # Genera la descripción del esquema y los embeddings de las columnas.
            self._generate_schema(df)

            return True, f"✅ Archivo cargado: {df.shape[0]} filas, {df.shape[1]} columnas."

        except Exception as e:
            return False, f"Error cargando archivo: {str(e)}"

    def _generate_schema(self, df):
        """Analiza el DataFrame para generar una descripción del esquema y embeddings de columnas."""
        schema_parts = []
        for col in df.columns:
            # Detecta el tipo de dato de cada columna.
            if pd.api.types.is_numeric_dtype(df[col]):
                col_type = 'NUMERIC'
                sample_values = f"rango: {df[col].min():.2f} - {df[col].max():.2f}"
            elif df[col].nunique() <= 20:
                col_type = 'CATEGORICAL'
                unique_vals = [str(val) for val in df[col].unique()[:5]]
                sample_values = f"valores: {', '.join(unique_vals)}"
            else:
                col_type = 'TEXT'
                sample_values = f"texto con {df[col].nunique()} valores únicos"

            # Crea una descripción para la columna y genera su embedding.
            desc = f"{col} ({col_type}): {sample_values}"
            self.column_info[col] = {'type': col_type, 'sample_values': sample_values}
            self.column_embeddings[col] = EMBEDDING_MODEL.encode([desc])[0]
            schema_parts.append(f"- {desc}")

        self.schema_description = f"Tabla: {TABLE_NAME}\nTotal de registros: {len(df)}\nColumnas:\n" + "\n".join(schema_parts)

    def find_relevant_columns(self, query, top_k=5):
        """Encuentra las columnas más relevantes para una consulta usando búsqueda semántica."""
        query_emb = EMBEDDING_MODEL.encode([query])[0]
        scores = [(col, cosine_similarity([query_emb], [emb])[0][0]) for col, emb in self.column_embeddings.items()]
        return sorted(scores, key=lambda x: x[1], reverse=True)[:top_k]

    def generate_sql(self, query):
        """Genera una consulta SQL a partir de la pregunta del usuario usando el LLM."""
        if not self.conn or not client:
            return "SELECT 1"

        relevant_cols_info = self.find_relevant_columns(query)
        cols_context = "\nColumnas más relevantes:\n" + "\n".join([f"- {col} ({self.column_info[col]['type']}): {self.column_info[col]['sample_values']}" for col, _ in relevant_cols_info])

        # El prompt guía al LLM para que genere un SQL correcto y seguro.
        prompt = f"""Eres un experto en SQL. Genera una consulta SQL para la pregunta del usuario.

ESQUEMA:
{self.schema_description}
{cols_context}

PREGUNTA: {query}

INSTRUCCIONES:
1. Usa el nombre de tabla: {TABLE_NAME}
2. Si la pregunta pide una lista de datos, usa 'LIMIT 1000' para proteger la memoria.
3. NO uses LIMIT para agregaciones (COUNT, SUM, AVG).
4. Responde SOLO con la consulta SQL.

CONSULTA SQL:"""

        try:
            # Llama a la API del LLM.
            response = client.chat.completions.create(model=GROQ_MODEL, messages=[{"role": "user", "content": prompt}], max_tokens=200, temperature=0.1)
            return response.choices[0].message.content.strip().replace('```sql', '').replace('```', '').strip()
        except Exception as e:
            print(f"Error generando SQL: {e}")
            return f"SELECT * FROM {TABLE_NAME} LIMIT 100" # Fallback seguro.

    def execute_query(self, sql):
        """Ejecuta la consulta SQL en la base de datos en memoria."""
        if not self.conn:
            return "Error: Base de datos no inicializada", pd.DataFrame()
        try:
            return sql, pd.read_sql_query(sql, self.conn)
        except Exception as e:
            # Si falla la consulta, intenta con una consulta de respaldo.
            fallback_sql = f"SELECT * FROM {TABLE_NAME} LIMIT 100"
            try:
                return fallback_sql, pd.read_sql_query(fallback_sql, self.conn)
            except Exception as e2:
                return f"Error Crítico: {e2}", pd.DataFrame()

    def generate_answer(self, query, sql_executed, data):
        """Genera una respuesta en lenguaje natural basada en los resultados de la consulta."""
        if not client or data.empty:
            return "No se encontraron datos o el servicio LLM no está disponible."

        # Prepara un resumen de los datos para dárselo como contexto al LLM.
        data_sample_str = data.head(15).to_string(index=False)
        if len(data) > 15:
            data_sample_str += f"\n... (mostrando 15 de {len(data)} registros)"

        prompt = f"""Analiza estos resultados y responde la pregunta del usuario de forma clara.

PREGUNTA ORIGINAL: {query}
DATOS OBTENIDOS (procesando {len(data)} registros):
{data_sample_str}

INSTRUCCIONES:
- Responde en español, basándote en los datos.
- Da números exactos y conclusiones directas.
- No repitas la pregunta.

RESPUESTA:"""
        try:
            # Llama a la API del LLM para generar la respuesta.
            response = client.chat.completions.create(model=GROQ_MODEL, messages=[{"role": "user", "content": prompt}], max_tokens=300, temperature=0.2)
            return response.choices[0].message.content.strip()
        except Exception as e:
            return f"Error generando respuesta: {e}"

    def create_chart(self, data):
        """Crea una gráfica de Plotly automáticamente basada en los tipos de datos."""
        if data.empty:
            return None

        # Usa una muestra si los datos son muy grandes para graficar.
        chart_data = data.sample(MAX_ROWS_FOR_ANALYSIS) if len(data) > MAX_ROWS_FOR_ANALYSIS else data

        try:
            numeric_cols = chart_data.select_dtypes(include=np.number).columns.tolist()
            categorical_cols = chart_data.select_dtypes(include=['object', 'category']).columns.tolist()
            fig = None

            # Lógica para decidir qué tipo de gráfica es más apropiada.
            if len(numeric_cols) >= 1 and len(categorical_cols) >= 1:
                grouped_data = data.groupby(categorical_cols[0])[numeric_cols[0]].sum().nlargest(20).reset_index()
                fig = px.bar(grouped_data, x=categorical_cols[0], y=numeric_cols[0], title=f"{numeric_cols[0]} por {categorical_cols[0]} (Top 20)")
            elif len(numeric_cols) >= 2:
                fig = px.scatter(chart_data, x=numeric_cols[0], y=numeric_cols[1], title=f"{numeric_cols[1]} vs {numeric_cols[0]}")
            elif len(numeric_cols) == 1:
                fig = px.histogram(chart_data, x=numeric_cols[0], title=f"Distribución de {numeric_cols[0]}")
            elif len(categorical_cols) >= 1:
                counts = data[categorical_cols[0]].value_counts().nlargest(20)
                fig = px.bar(x=counts.index, y=counts.values, title=f"Frecuencia de {categorical_cols[0]} (Top 20)")

            if fig:
                fig.update_layout(height=400, template="plotly_white")
                return fig
        except Exception as e:
            print(f"Error creando gráfica: {e}")
        return None

    def should_create_chart(self, query):
        """Determina si la consulta del usuario sugiere la creación de una gráfica."""
        chart_keywords = ['gráfica', 'grafica', 'gráfico', 'grafico', 'chart', 'plot', 'visualiza', 'muestra', 'distribución', 'tendencia']
        return any(keyword in query.lower() for keyword in chart_keywords)

# --- 5. Funciones de la Interfaz Gradio ---
# Estas funciones conectan la lógica del backend (DatabaseRAG) con los componentes de la UI.

def process_file_upload(file, rag_state):
    """Maneja el evento de carga de un archivo."""
    if file is None:
        return "Por favor, sube un archivo.", gr.update(interactive=False), rag_state

    # Crea una nueva instancia del motor RAG para esta sesión.
    rag_state = DatabaseRAG()
    success, message = rag_state.load_file(file.name)

    if success:
        return message + "\n\n" + rag_state.schema_description, gr.update(interactive=True), rag_state
    else:
        return message, gr.update(interactive=False), rag_state

def process_user_query(message, history, rag_state):
    """
    Orquesta el flujo completo: SQL -> Ejecución -> Respuesta -> Gráfica.
    NOTA: Esta función ya no limpia el cuadro de texto. Eso se hace en un evento .then() separado.
    """
    if not rag_state or not rag_state.conn:
        history.append([message, "Por favor, primero sube un archivo de datos."])
        return history, None, gr.update(visible=False)

    try:
        # Ejecuta el pipeline completo.
        sql_query = rag_state.generate_sql(message)
        sql_executed, data = rag_state.execute_query(sql_query)
        text_response = rag_state.generate_answer(message, sql_executed, data)
        history.append([message, text_response])

        # Crea una gráfica si es necesario.
        chart = rag_state.create_chart(data) if rag_state.should_create_chart(message) else None

        # Devuelve 3 salidas: historial del chat, la figura del gráfico y la actualización de visibilidad del gráfico.
        return history, chart, gr.update(visible=chart is not None)
    except Exception as e:
        history.append([message, f"Ocurrió un error: {e}"])
        return history, None, gr.update(visible=False)

# --- 6. Creación de la Interfaz de Usuario ---
def create_interface():
    """Define y organiza todos los componentes visuales de la aplicación Gradio."""
    with gr.Blocks(title="RAG Database Chatbot", theme=gr.themes.Soft()) as app:
        gr.Markdown("# 🤖 RAG Database Chatbot")
        gr.Markdown("Sube tu archivo (CSV, XLSX, JSON) y haz preguntas en lenguaje natural.")

        # Almacena el estado (la instancia de DatabaseRAG) para cada sesión de usuario.
        rag_state = gr.State(DatabaseRAG())

        # Define la disposición de los componentes (columnas, filas, etc.).
        with gr.Row():
            with gr.Column(scale=1):
                file_input = gr.File(label="📁 Cargar Archivo", file_types=[".csv", ".xlsx", ".xls", ".json"])
                file_status = gr.Textbox(label="Estado del Archivo", interactive=False, max_lines=10)

            with gr.Column(scale=2):
                chatbot = gr.Chatbot(label="Conversación", height=400, show_copy_button=True, avatar_images=("user.png", "bot.png"))
                msg_input = gr.Textbox(label="Tu pregunta", placeholder="Ej: ¿Cuáles son los 5 productos más vendidos?", interactive=False)
                with gr.Row():
                    send_btn = gr.Button("Enviar", variant="primary")
                    clear_btn = gr.Button("Limpiar")

        chart_output = gr.Plot(label="📊 Gráfica Automática", visible=False)

        # --- 7. Conexión de Eventos ---
        # Asocia las funciones a las acciones del usuario (clics, subidas, etc.).

        file_input.change(fn=process_file_upload, inputs=[file_input, rag_state], outputs=[file_status, msg_input, rag_state])

        # Define los disparadores para enviar una pregunta (Enter en el textbox o clic en el botón).
        submit_triggers = [msg_input.submit, send_btn.click]

        for trigger in submit_triggers:
            # 1. Ejecuta la lógica principal. La salida de esta función va al chatbot y a la gráfica.
            main_process = trigger(
                fn=process_user_query,
                inputs=[msg_input, chatbot, rag_state],
                outputs=[chatbot, chart_output, chart_output] # Salidas: chatbot, datos del gráfico, visibilidad del gráfico
            )
            # 2. Encadena una acción para limpiar el cuadro de texto.
            main_process.then(fn=lambda: "", inputs=None, outputs=[msg_input], queue=False)


        def clear_chat():
            # Limpia todos los componentes relevantes de la interfaz.
            return [], "", None, gr.update(visible=False)

        clear_btn.click(
            fn=clear_chat,
            outputs=[chatbot, msg_input, chart_output, chart_output]
        )

    return app

# --- 8. Lanzamiento de la Aplicación ---
if __name__ == "__main__":
    # Crea la interfaz y la lanza.
    app = create_interface()
    app.launch(share=True, show_error=True, debug=True)